<h2>Knn Regression</h2>

In [2]:
import numpy as np
import pandas as pd
import math as ma
import matplotlib.pyplot as plt

<h3>Define helper functions</h3>

In [46]:
def get_csv(path: str) -> pd.DataFrame:
    
    df = pd.read_csv(path, parse_dates = ['Date'])\
        .drop(columns = ['Currency', '24h Open (USD)'])\
        .rename(columns = {'Date': 'date', 'Closing Price (USD)': 'price', '24h High (USD)': 'high', '24h Low (USD)': 'low'})

    return df


def rolling_forecast(df: pd.DataFrame, test_ratio: float = 0.3) -> pd.DataFrame:

    prediction_roll = pd.Series(dtype = 'float64')
    test_data = df[ma.floor(len(df.index) * (1 - test_ratio)) + 1:]

    for end_date in test_data.index:
        train_data = df[:end_date - 1]
        # TODO: define knn-regression model
        pred = pd.mean(train_data)
        prediction_roll.loc[end_date] = pred

    return pd.DataFrame({'pred': prediction_roll, 'real': test_data['price']})

<h3>Import data</h3>

In [47]:
df_ada = get_csv('./data/Cardano.csv')
df_eth = get_csv('./data/ethereum.csv')
df_sol = get_csv('./data/solana.csv')
df_doge = get_csv('./data/dogecoin.csv')

<h3>Problems with using knn</h3>
<p>
    Knn regression is useful when we want to predict some values of an object based on that object other values. However, when it comes to predicting the prices of tomorrow's cryptos, there's a problem: we don't know anything about the tomorrow's, so instead, we should infer the price of tomorrow's cryptos using today's data.
</p>

In [48]:
def transform_df(df: pd.DataFrame) -> pd.DataFrame:

    transformed_df = pd.DataFrame(columns = ['date', 'price', 'previous price', 'previous high', 'previous low'])

    for i in range(1, df.shape[0]):
        transformed_df.loc[i] = {'date': df.loc[i]['date'], 'price': df.loc[i]['price'], 'previous price': df.loc[i - 1]['price'], 'previous high': df.loc[i - 1]['high'], 'previous low': df.loc[i - 1]['low']}

    return transformed_df